In [ ]:
import epxexec # Epistemix package for running FRED simulations within Python
import epxexec.epxresults as epxresults # Epistemix package for managing and accessing FRED simulation outputs
from epxexec.fred_job import fred_job
import pandas as pd

In [ ]:
import os
from pathlib import Path
os.environ["FRED_PROJECT"] = str(Path.cwd().parent.parent)

# Ground logistics model

This model represents a ground transportation network. Cities represent nodes in the network and roads are edges. A table of trucks agents is read in at the beginning of the simulation. There are no human agents in the model. 

## Model components

- `city_locations.fred`: contains the GENERATE\_LOCATIONS condition that creates a network of cities
- `trucks.fred`: contains the GENERATE\_TRUCKS condition that reads the trucks agent file and the MOVE\_TRUCKS condition that picks next locations for trucks to travel to
- `travel_disruption.fred`: contains the DISRUPTION condition that creates travel disruptions at certain nodes as defined by an end user

In [ ]:
ground_logistics = fred_job(program="main_GROUND_LOGISTICS.fred")

## Data outputs

Let's take a look at the output `travel_info.csv`. The data include the distance in miles and time in days traveled by each truck each between destinations. 

In [ ]:
travel_log_df = ground_logistics.runs[1].get_csv_output('travel_log.csv')
travel_log_df